In [166]:
import protein
from protein import Protein, trypsin

import measurement
from measurement import read_mgf

import pandas as pd
from multiprocessing import Pool
import numpy as np

import importlib
from pprint import pprint

importlib.reload(measurement)
importlib.reload(protein)

<module 'protein' from '/Users/eugen/code/bp/bp-code/src/protein.py'>

In [146]:
OVA = "GSIGAASMEFCFDVFKELKVHHANENIFYCPIAIMSALAMVYLGAKDSTRTQINKVVRFDKLPGFGDSIEAQCGTSVNVHSSLRDILNQITKPNDVYSFSLASRLYAEERYPILPEYLQCVKELYRGGLEPINFQTAADQARELINSWVESQTNGIIRNVLQPSSVDSQTAMVLVNAIVFKGLWEKAFKDEDTQAMPFRVTEQESKPVQMMYQIGLFRVASMASEKMKILELPFASGTMSMLVLLPDEVSGLEQLESIINFEKLTEWTSSNVMEERKIKVYLPRMKMEEKYNLTSVLMAMGITDVFSSSANLSGISSAESLKISQAVHAAHAEINEAGREVVGSAEAGVDAASVSEEFRADHPFLFCIKHIATNAVLFFGRCVSP"
LYS = "KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL"
BSA = "DTHKSEIAHRFKDLGEEHFKGLVLIAFSQYLQQCPFDEHVKLVNELTEFAKTCVADESHAGCEKSLHTLFGDELCKVASLRETYGDMADCCEKQEPERNECFLSHKDDSPDLPKLKPDPNTLCDEFKADEKKFWGKYLYEIARRHPYFYAPELLYYANKYNGVFQECCQAEDKGACLLPKIETMREKVLTSSARQRLRCASIQKFGERALKAWSVARLSQKFPKAEFVEVTKLVTDLTKVHKECCHGDLLECADDRADLAKYICDNQDTISSKLKECCDKPLLEKSHCIAEVEKDAIPENLPPLTADFAEDKDVCKNYQEAKDAFLGSFLYEYSRRHPEYAVSVLLRLAKEYEATLEECCAKDDPHACYSTVFDKLKHLVDEPQNLIKQNCDQFEKLGEYGFQNALIVRYTRKVPQVSTPTLVEVSRSLGKVGTRCCTKPESERMPCTEDYLSLILNRLCVLHEKTPVSEKVTKCCTESLVNRRPCFSALTPDETYVPKAFDEKLFTFHADICTLPDTEKQIKKQTALVELLKHKPKATEEQLKTVMENFVAFVDKCCAADDKEACFAVEGPKLVVSTQTALA"

# LYS můstky
# VFGRCELAAA + WIRGCRL
# GNWVCAAKFE + WRNRCKGTDV
# SRWWCNDGRT + CNIPCSALLS
# SRNLCNIPCS + ASVNCAKKIV

# TODO
# Protein Pilot
# Paragon
# AA tagy ze spektra

In [122]:
protein = Protein(LYS)
peptides = list(protein.digest(trypsin))
measurements = list(read_mgf("../data/mgf/190318_LYS_RAT_50x_05.mgf"))

In [173]:
tolerance_ppm_precursor = 10
multiplier = measurement.binary_multiplier(tolerance_ppm=10)

result = []
peps_with_threshold = [
    (pep, (tolerance_ppm_precursor / 1e6) * pep.total_mz) for pep in peptides
]
for i, m in enumerate(measurements):
    if i % 500 == 0:
        print(f"Done: {i}")

    for pepid, (pep, t) in enumerate(peps_with_threshold):
        if pep.total_charge == m.charge and abs(pep.total_mz - m.peptide_mz) <= t:
            score = m.score_match(pep, multiplier)

            result.append(
                {
                    "peptide": str(pep),
                    "is_valid": pep.ismerged or len(pep.peptides[0].seq) > 5,
                    "score": score,
                    "mod": pep.modstr,
                    "scan": m.scan,
                    "mass_error": pep.total_mz - m.peptide_mz,
                    "measurement": i,
                    "pepid": pepid
                    # "silico_frags": ";".join(str(m) for m in pep.fragment_masses()),
                    # "frags": ";".join(str(f) for f in m.fragments_mz),
                    # "frags_intensity": ";".join(str(f) for f in m.fragments_intensity)
                }
            )

# Nechat pokud
# má vysokou pnost
# nebo obsahuje cystein a je kolem něj díra, a jinak má dobré pokrytí
# nebo je krátký a obsahuje cystein, pak na pokrytí nezáleží

# a u těchto se podívat na to, co v tom spektru přebývá?

Done: 0
Done: 500
Done: 1000
Done: 1500
Done: 2000
Done: 2500
Done: 3000
Done: 3500
Done: 4000
Done: 4500
Done: 5000
Done: 5500
Done: 6000
Done: 6500
Done: 7000
Done: 7500
Done: 8000
Done: 8500
Done: 9000
Done: 9500
Done: 10000
Done: 10500
Done: 11000
Done: 11500
Done: 12000
Done: 12500
Done: 13000
Done: 13500
Done: 14000
Done: 14500


In [174]:
df = pd.DataFrame(result)
df.to_csv("../out/lys_analysis_rat.csv", index=False)


In [125]:

def show_match(
    msmnt, peptide, multiplier=measurement.binary_multiplier(tolerance_ppm=10)
):
    if peptide.ismerged:
        generated = peptide.noncysteine_fragment_masses
    else:
        generated = peptide.fragment_masses

    indices = np.searchsorted(msmnt.masses_padded, generated)
    left = multiplier(generated, msmnt.masses_padded[indices - 1])
    right = multiplier(generated, msmnt.masses_padded[indices])

    matched_intensities = np.maximum(
        left * msmnt.intensities_padded[indices - 1],
        right * msmnt.intensities_padded[indices],
    )

    matched_masses = np.select(
        [
            (left == 0) & (right == 0),
            left * msmnt.intensities_padded[indices - 1]
            > right * msmnt.intensities_padded[indices],
            True,
        ],
        [
            0,
            msmnt.masses_padded[indices - 1],
            msmnt.masses_padded[indices],
        ],
    )

    nonzero = np.nonzero(matched_masses)[0]

    fragments = [peptide.fragment()[i] for i in nonzero]
    intesities_m = matched_intensities[nonzero]

    return {
        "fragments": [
            {
                "name": f.name,
                "mass_theoretical": f.mz,
                "mass_measured": matched_masses[nonzero][i],
                "intensity": round(intesities_m[i], 4),
                "intensity_relative": round(
                    100 * intesities_m[i] / np.sum(msmnt.fragments_intensity), 2
                ),
            }
            for i, f in enumerate(fragments)
        ],
        "score": round(
            100 * np.sum(intesities_m) / np.sum(msmnt.fragments_intensity),
            ndigits=2,
        ),
    }


m = measurements[8237]
p = peptides[2355]
pprint(show_match(m, p))

{'fragments': [{'intensity': 2270.7703,
                'intensity_relative': 0.82,
                'mass_measured': 175.1191133,
                'mass_theoretical': 175.118952174959,
                'name': 'y1[+]'},
               {'intensity': 4214.5146,
                'intensity_relative': 1.52,
                'mass_measured': 1152.546051,
                'mass_theoretical': 1152.539225047239,
                'name': 'y10[+]'},
               {'intensity': 4035.3948,
                'intensity_relative': 1.45,
                'mass_measured': 866.9081289,
                'mass_theoretical': 866.913152589629,
                'name': 'b15[2+]'},
               {'intensity': 3363.0364,
                'intensity_relative': 1.21,
                'mass_measured': 1146.540068,
                'mass_theoretical': 1146.5419649771688,
                'name': '[b20-H2O][2+]'},
               {'intensity': 2195.3447,
                'intensity_relative': 0.79,
                'mass_measured

In [103]:
print(
    [
        f"{f.name} | {f.mz}"
        for f in sorted(
            peptides[8435].fragment(),
            key=lambda f: f.index if f.index is not None else 0,
        )
        if f.index == 20
    ]
)

['b20[+] | 2310.087218171489', '[b20-NH3][+] | 2293.060669070369', '[b20-H2O][+] | 2292.0766534874588', 'y20[+] | 2315.0621256219692', '[y20-NH3][+] | 2298.035576520849', '[y20-H2O][+] | 2297.051560937939', 'b20[2+] | 1155.547247319184', '[b20-NH3][2+] | 1147.033972768624', '[b20-H2O][2+] | 1146.5419649771688', 'y20[2+] | 1158.034701044424', '[y20-NH3][2+] | 1149.521426493864', '[y20-H2O][2+] | 1149.029418702409', 'b20[3+] | 770.7005903684156', '[b20-NH3][3+] | 765.0250740013756', '[b20-H2O][3+] | 764.6970688070722', 'y20[3+] | 772.358892851909', '[y20-NH3][3+] | 766.683376484869', '[y20-H2O][3+] | 766.3553712905656', 'b20[4+] | 578.2772618930316', '[b20-NH3][4+] | 574.0206246177515', '[b20-H2O][4+] | 573.774620722024', 'y20[4+] | 579.5209887556516', '[y20-NH3][4+] | 575.2643514803716', '[y20-H2O][4+] | 575.0183475846441']


In [172]:
sorted([str(p) for p in peptides if str(p).startswith("FES")])

['FESNFNTQATNR',
 'FESNFNTQATNR',
 'FESNFNTQATNR',
 'FESNFNTQATNR',
 'FESNFNTQATNR',
 'FESNFNTQATNRNTDGSTDYGILQINSR',
 'FESNFNTQATNRNTDGSTDYGILQINSR',
 'FESNFNTQATNRNTDGSTDYGILQINSR',
 'FESNFNTQATNRNTDGSTDYGILQINSR',
 'FESNFNTQATNRNTDGSTDYGILQINSR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CK',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CK',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CK',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CK',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CK',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CK',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CKGTDVQAWIR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CKGTDVQAWIR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CKGTDVQAWIR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CKGTDVQAWIR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR+CKGTDVQAWIR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWW

In [141]:
# FESNFNTQATNRNTDGSTDYGILQINSR
peptides_initial = trypsin(LYS)
peptides_skip = {
    (peptides_initial[i][0], peptides_initial[i + s][1])
    for s in range(2 + 1)
    for i in range(len(peptides_initial) - s)
}
sorted([LYS[b:e] for b, e in peptides_skip])

['CELAAAMK',
 'CELAAAMKR',
 'CELAAAMKRHGLDNYR',
 'CK',
 'CKGTDVQAWIR',
 'CKGTDVQAWIRGCR',
 'FESNFNTQATNR',
 'FESNFNTQATNRNTDGSTDYGILQINSR',
 'FESNFNTQATNRNTDGSTDYGILQINSRWWCNDGR',
 'GCR',
 'GCRL',
 'GTDVQAWIR',
 'GTDVQAWIRGCR',
 'GTDVQAWIRGCRL',
 'GYSLGNWVCAAK',
 'GYSLGNWVCAAKFESNFNTQATNR',
 'GYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSR',
 'HGLDNYR',
 'HGLDNYRGYSLGNWVCAAK',
 'HGLDNYRGYSLGNWVCAAKFESNFNTQATNR',
 'IVSDGNGMNAWVAWR',
 'IVSDGNGMNAWVAWRNR',
 'IVSDGNGMNAWVAWRNRCK',
 'K',
 'K',
 'KIVSDGNGMNAWVAWR',
 'KIVSDGNGMNAWVAWRNR',
 'KVFGR',
 'KVFGRCELAAAMK',
 'L',
 'NLCNIPCSALLSSDITASVNCAK',
 'NLCNIPCSALLSSDITASVNCAKK',
 'NLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWR',
 'NR',
 'NRCK',
 'NRCKGTDVQAWIR',
 'NTDGSTDYGILQINSR',
 'NTDGSTDYGILQINSRWWCNDGR',
 'NTDGSTDYGILQINSRWWCNDGRTPGSR',
 'R',
 'RHGLDNYR',
 'RHGLDNYRGYSLGNWVCAAK',
 'TPGSR',
 'TPGSRNLCNIPCSALLSSDITASVNCAK',
 'TPGSRNLCNIPCSALLSSDITASVNCAKK',
 'VFGR',
 'VFGRCELAAAMK',
 'VFGRCELAAAMKR',
 'WWCNDGR',
 'WWCNDGRTPGSR',
 'WWCNDGRTPGSRNLCNIPCSAL